# Categorical Feature Encoding Challenge II

## Keras - 1 Attempt

## Attempt 1

In [1]:
import os, random, time, json
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy

# Reproducibility
os.environ['PYTHONHASHSEED'] = '42'
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Load data
train_files = ['cat-in-the-dat-ii/train.csv.zip']
df_list = [pd.read_csv(f) for f in train_files]
df = pd.concat(df_list, ignore_index=True)
df_test = pd.read_csv('cat-in-the-dat-ii/test.csv.zip')

# Infer ids and targets
id_col = 'id'
target_columns = ['target']

# Encode target
le = LabelEncoder().fit(df[target_columns[0]].astype(str))
y_train = le.transform(df[target_columns[0]].astype(str)).astype(int)

# Prepare features
X_train = df.drop(columns=target_columns + [id_col], errors='ignore')
X_test = df_test.drop(columns=target_columns + [id_col], errors='ignore')
test_ids = df_test[id_col]

# Feature engineering: drop all-missing columns
missing_cols = X_train.columns[X_train.isna().all()].tolist()
X_train = X_train.drop(columns=missing_cols)
X_test = X_test.drop(columns=missing_cols)

# Identify numeric and categorical
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

# Remove high-cardinality categoricals
low_cardinality = [c for c in categorical_features if X_train[c].nunique() <= 50]
high_card_cols = [c for c in categorical_features if c not in low_cardinality]
X_train = X_train.drop(columns=high_card_cols)
X_test = X_test.drop(columns=high_card_cols)
categorical_features = low_cardinality

# Preprocessing pipelines
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Fit and transform data
X_train_proc = preprocessor.fit_transform(X_train)
X_test_proc = preprocessor.transform(X_test)

# Model building
n_samples, n_features = X_train_proc.shape
inputs = Input(shape=(n_features,))

if n_samples < 10000 or n_features < 100:
    units = [min(n_features*2, 128), min(n_features, 64)]
    x = inputs
    for u in units:
        x = Dense(int(u), activation='relu')(x)
        x = Dropout(0.3)(x)
else:
    sizes = [min(n_features*i, 1024) for i in (2, 1, 0.5, 0.25)]
    sizes = [int(s) for s in sizes if s >= 16]
    x = inputs
    for u in sizes:
        x = Dense(u, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.3)(x)

outputs = Dense(1, activation='sigmoid')(x)
model = Model(inputs, outputs)
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
]

# Training
start_time = time.time()
history = model.fit(
    X_train_proc, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=128,
    callbacks=callbacks,
    verbose=2
)
duration = time.time() - start_time

# Logging results
last = len(history.history['loss']) - 1
results = {
    'training_accuracy': float(history.history['accuracy'][last]),
    'training_loss': float(history.history['loss'][last]),
    'validation_accuracy': float(history.history['val_accuracy'][last]),
    'validation_loss': float(history.history['val_loss'][last])
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# Prediction & Submission
raw_preds = model.predict(X_test_proc)
probs = raw_preds.flatten()
final = probs

# Ensure correct shape
final = final.reshape(-1, 1)
submission = pd.DataFrame(final, columns=target_columns)
submission.insert(0, id_col, test_ids.reset_index(drop=True))
submission.to_csv('submission_result.csv', index=False)

2025-07-09 22:45:58.620654: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752101158.637458  930396 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752101158.642520  930396 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752101158.656926  930396 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752101158.656939  930396 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752101158.656941  930396 computation_placer.cc:177] computation placer alr

Epoch 1/100


3750/3750 - 8s - 2ms/step - accuracy: 0.8144 - auc: 0.7179 - loss: 0.4359 - val_accuracy: 0.8165 - val_auc: 0.7345 - val_loss: 0.4265
Epoch 2/100


3750/3750 - 7s - 2ms/step - accuracy: 0.8157 - auc: 0.7302 - loss: 0.4299 - val_accuracy: 0.8166 - val_auc: 0.7357 - val_loss: 0.4258
Epoch 3/100
3750/3750 - 7s - 2ms/step - accuracy: 0.8157 - auc: 0.7328 - loss: 0.4287 - val_accuracy: 0.8166 - val_auc: 0.7355 - val_loss: 0.4260
Epoch 4/100
3750/3750 - 7s - 2ms/step - accuracy: 0.8158 - auc: 0.7339 - loss: 0.4282 - val_accuracy: 0.8167 - val_auc: 0.7356 - val_loss: 0.4259
Epoch 5/100
3750/3750 - 7s - 2ms/step - accuracy: 0.8162 - auc: 0.7352 - loss: 0.4275 - val_accuracy: 0.8169 - val_auc: 0.7355 - val_loss: 0.4263
Epoch 6/100
3750/3750 - 6s - 2ms/step - accuracy: 0.8157 - auc: 0.7360 - loss: 0.4272 - val_accuracy: 0.8165 - val_auc: 0.7354 - val_loss: 0.4262
Epoch 7/100
3750/3750 - 6s - 2ms/step - accuracy: 0.8160 - auc: 0.7366 - loss: 0.4269 - val_accuracy: 0.8166 - val_auc: 0.7353 - val_loss: 0.4261
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 7s 575us/step


## Keras Tuner - 2 Attempt

## Attempt 1

In [ ]:
import os, random, time, json
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy
import keras_tuner as kt

# Reproducibility
os.environ['PYTHONHASHSEED'] = '42'
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Load data
train_files = ['cat-in-the-dat-ii/train.csv.zip']
df_list = [pd.read_csv(f) for f in train_files]
df = pd.concat(df_list, ignore_index=True)
df_test = pd.read_csv('cat-in-the-dat-ii/test.csv.zip')

# Infer ids and targets
id_col = 'id'
target_columns = ['target']

# Encode target
le = LabelEncoder().fit(df[target_columns[0]].astype(str))
y_train = le.transform(df[target_columns[0]].astype(str)).astype(int)

# Prepare features
X_train = df.drop(columns=target_columns + [id_col], errors='ignore')
X_test = df_test.drop(columns=target_columns + [id_col], errors='ignore')
test_ids = df_test[id_col]

# Feature engineering: drop all-missing columns
missing_cols = X_train.columns[X_train.isna().all()].tolist()
X_train = X_train.drop(columns=missing_cols)
X_test = X_test.drop(columns=missing_cols)

# Identify numeric and categorical
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

# Remove high-cardinality categoricals
low_cardinality = [c for c in categorical_features if X_train[c].nunique() <= 50]
high_card_cols = [c for c in categorical_features if c not in low_cardinality]
X_train = X_train.drop(columns=high_card_cols)
X_test = X_test.drop(columns=high_card_cols)
categorical_features = low_cardinality

# Preprocessing pipelines
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Fit and transform data
X_train_proc = preprocessor.fit_transform(X_train)
X_test_proc = preprocessor.transform(X_test)

# Model building
n_samples, n_features = X_train_proc.shape
inputs = Input(shape=(n_features,))

# Keras-Tuner model definition
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        layers = hp.Int('layers', 2, 8)
        units = hp.Int('units', 64, 1024, step=64)
        act = hp.Choice('activation', ['relu'])
        drop = hp.Float('dropout', 0.0, 0.5, step=0.1)
        opt = hp.Choice('optimizer', ['adam'])
        lr = hp.Float('learning_rate', 1e-5, 0.01, sampling='log')

        inputs = Input(shape=(n_features,))
        x = inputs
        for _ in range(layers):
            x = Dense(units, activation=act)(x)
            x = Dropout(drop)(x)
        outputs = Dense(1, activation='sigmoid')(x)
        model = Model(inputs, outputs)
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])
        return model

# Tuner setup
bs = 32  # Example batch size
ep = 20   # Example epochs

tuner = kt.BayesianOptimization(
    MyHyperModel(),
    objective='val_loss',
    max_trials=1,
    executions_per_trial=1,
    seed=42,
    overwrite=True,
    project_name='bayesian_tuner'
)

if y_val is not None:
    tuner.search(
        X_train_proc, y_train,
        validation_data=(X_val_proc, y_val),
        batch_size=1, epochs=ep,
        callbacks=[early_stopping, checkpoint]
    )
else:
    tuner.search(
        X_train_proc, y_train,
        validation_split=0.2,
        batch_size=1, epochs=ep,
        callbacks=[early_stopping, checkpoint]
    )

model = tuner.hypermodel.build(
    tuner.get_best_hyperparameters(1)[0]
)

# Retrain model with original callbacks and data
if y_val is not None:
    history = model.fit(
        X_train_proc, y_train,
        validation_data=(X_val_proc, y_val),
        epochs=1, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )
else:
    history = model.fit(
        X_train_proc, y_train,
        validation_split=0.2,
        epochs=1, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
]

# Training
start_time = time.time()
history = model.fit(
    X_train_proc, y_train,
    validation_split=0.2,
    epochs=1,
    batch_size=128,
    callbacks=callbacks,
    verbose=2
)
duration = time.time() - start_time

# Logging results
last = len(history.history['loss']) - 1
results = {
    'training_accuracy': float(history.history['accuracy'][last]),
    'training_loss': float(history.history['loss'][last]),
    'validation_accuracy': float(history.history['val_accuracy'][last]),
    'validation_loss': float(history.history['val_loss'][last])
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# Prediction & Submission
raw_preds = model.predict(X_test_proc)
probs = raw_preds.flatten()
final = probs

# Ensure correct shape
final = final.reshape(-1, 1)
submission = pd.DataFrame(final, columns=target_columns)
submission.insert(0, id_col, test_ids.reset_index(drop=True))
submission.to_csv('submission_result.csv', index=False)

2025-07-12 23:56:51.704800: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752364611.721291 1095551 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752364611.726308 1095551 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752364611.740459 1095551 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752364611.740474 1095551 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752364611.740477 1095551 computation_placer.cc:177] computation placer alr

NameError: name 'y_val' is not defined

## Attempt 2

In [ ]:
import os, random, time, json
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy
import keras_tuner as kt

# Reproducibility
os.environ['PYTHONHASHSEED'] = '42'
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Load data
train_files = ['train.csv.zip']
df_list = [pd.read_csv(f) for f in train_files]
df = pd.concat(df_list, ignore_index=True)
df_test = pd.read_csv('test.csv.zip')

# Infer ids and targets
id_col = 'id'
target_columns = ['target']

# Encode target
le = LabelEncoder().fit(df[target_columns[0]].astype(str))
y_train = le.transform(df[target_columns[0]].astype(str)).astype(int)

# Prepare features
X_train = df.drop(columns=target_columns + [id_col], errors='ignore')
X_test = df_test.drop(columns=target_columns + [id_col], errors='ignore')
test_ids = df_test[id_col]

# Feature engineering: drop all-missing columns
missing_cols = X_train.columns[X_train.isna().all()].tolist()
X_train = X_train.drop(columns=missing_cols)
X_test = X_test.drop(columns=missing_cols)

# Identify numeric and categorical
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

# Remove high-cardinality categoricals
low_cardinality = [c for c in categorical_features if X_train[c].nunique() <= 50]
high_card_cols = [c for c in categorical_features if c not in low_cardinality]
X_train = X_train.drop(columns=high_card_cols)
X_test = X_test.drop(columns=high_card_cols)
categorical_features = low_cardinality

# Preprocessing pipelines
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Fit and transform data
X_train_proc = preprocessor.fit_transform(X_train)
X_test_proc = preprocessor.transform(X_test)

# Model building
n_samples, n_features = X_train_proc.shape
inputs = Input(shape=(n_features,))

# Keras-Tuner model definition
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        layers = hp.Int('layers', 2, 8)
        units = hp.Int('units', 64, 1024, step=64)
        act = hp.Choice('activation', ['relu'])
        drop = hp.Float('dropout', 0.0, 0.5, step=0.1)
        opt = hp.Choice('optimizer', ['adam'])
        lr = hp.Float('learning_rate', 1e-5, 0.01, sampling='log')

        inputs = Input(shape=(n_features,))
        x = inputs
        for _ in range(layers):
            x = Dense(units, activation=act)(x)
            x = Dropout(drop)(x)
        outputs = Dense(1, activation='sigmoid')(x)
        model = Model(inputs, outputs)
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])
        return model

# Tuner setup
bs = 32  # Example batch size
ep = 20   # Example epochs

tuner = kt.BayesianOptimization(
    MyHyperModel(),
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    seed=42,
    overwrite=False,
    project_name='bayesian_tuner'
)

# Split data for validation
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_proc, y_train, test_size=0.2, random_state=42)

tuner.search(
    X_train_split, y_train_split,
    validation_data=(X_val_split, y_val_split),
    batch_size=bs, epochs=ep,
    callbacks=[early_stopping, checkpoint]
)

model = tuner.hypermodel.build(
    tuner.get_best_hyperparameters(1)[0]
)

start_time = time.time()

# Retrain model with original callbacks and data
history = model.fit(
    X_train_proc, y_train,
    validation_split=0.2,
    epochs=100, batch_size=bs,
    callbacks=[early_stopping, checkpoint],
    verbose=2
)

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
]

# Training
history = model.fit(
    X_train_proc, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=128,
    callbacks=callbacks,
    verbose=2
)
duration = time.time() - start_time

# Logging results
last = len(history.history['loss']) - 1
results = {
    'training_accuracy': float(history.history['accuracy'][last]),
    'training_loss': float(history.history['loss'][last]),
    'validation_accuracy': float(history.history['val_accuracy'][last]),
    'validation_loss': float(history.history['val_loss'][last])
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# Prediction & Submission
raw_preds = model.predict(X_test_proc)
probs = raw_preds.flatten()
final = probs

# Ensure correct shape
final = final.reshape(-1, 1)
submission = pd.DataFrame(final, columns=target_columns)
submission.insert(0, id_col, test_ids.reset_index(drop=True))
submission.to_csv('submission_result.csv', index=False)

Reloading Tuner from ./bayesian_tuner/tuner0.json


I0000 00:00:1752762763.074851 3942539 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 37024 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:04:00.0, compute capability: 8.0


Epoch 1/100


I0000 00:00:1752762767.030410 3944212 service.cc:152] XLA service 0x723c00003a40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752762767.030513 3944212 service.cc:160]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB, Compute Capability 8.0
2025-07-17 14:32:47.133724: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1752762767.523375 3944212 cuda_dnn.cc:529] Loaded cuDNN version 90501
2025-07-17 14:32:49.316536: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_358', 56 bytes spill stores, 56 bytes spill loads

2025-07-17 14:32:49.746624: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_687', 12

15000/15000 - 80s - 5ms/step - accuracy: 0.8146 - auc: 0.7270 - loss: 0.4316 - val_accuracy: 0.8166 - val_auc: 0.7341 - val_loss: 0.4265
Epoch 2/100
15000/15000 - 58s - 4ms/step - accuracy: 0.8160 - auc: 0.7338 - loss: 0.4280 - val_accuracy: 0.8166 - val_auc: 0.7351 - val_loss: 0.4270
Epoch 3/100
15000/15000 - 66s - 4ms/step - accuracy: 0.8164 - auc: 0.7359 - loss: 0.4269 - val_accuracy: 0.8169 - val_auc: 0.7351 - val_loss: 0.4272
Epoch 4/100
15000/15000 - 68s - 5ms/step - accuracy: 0.8168 - auc: 0.7373 - loss: 0.4260 - val_accuracy: 0.8165 - val_auc: 0.7345 - val_loss: 0.4276
Epoch 5/100
15000/15000 - 57s - 4ms/step - accuracy: 0.8173 - auc: 0.7388 - loss: 0.4252 - val_accuracy: 0.8166 - val_auc: 0.7343 - val_loss: 0.4282
Epoch 6/100
15000/15000 - 56s - 4ms/step - accuracy: 0.8176 - auc: 0.7402 - loss: 0.4244 - val_accuracy: 0.8168 - val_auc: 0.7337 - val_loss: 0.4283
Epoch 7/100
15000/15000 - 54s - 4ms/step - accuracy: 0.8180 - auc: 0.7415 - loss: 0.4236 - val_accuracy: 0.8165 - val_

2025-07-17 14:44:01.449618: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_354', 136 bytes spill stores, 136 bytes spill loads

2025-07-17 14:44:01.580844: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_354', 364 bytes spill stores, 364 bytes spill loads

2025-07-17 14:44:02.264960: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_354', 64 bytes spill stores, 64 bytes spill loads

2025-07-17 14:44:02.320257: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_738', 152 bytes spill stores, 288 bytes spill loads

2025-07-17 14:44:02.441850: I external

3750/3750 - 24s - 7ms/step - accuracy: 0.8163 - auc: 0.7365 - loss: 0.4265 - val_accuracy: 0.8168 - val_auc: 0.7357 - val_loss: 0.4257
Epoch 2/100
3750/3750 - 16s - 4ms/step - accuracy: 0.8167 - auc: 0.7376 - loss: 0.4258 - val_accuracy: 0.8167 - val_auc: 0.7356 - val_loss: 0.4257
Epoch 3/100
3750/3750 - 16s - 4ms/step - accuracy: 0.8172 - auc: 0.7386 - loss: 0.4252 - val_accuracy: 0.8167 - val_auc: 0.7351 - val_loss: 0.4258
Epoch 4/100
3750/3750 - 17s - 5ms/step - accuracy: 0.8174 - auc: 0.7399 - loss: 0.4244 - val_accuracy: 0.8164 - val_auc: 0.7342 - val_loss: 0.4264
Epoch 5/100
3750/3750 - 16s - 4ms/step - accuracy: 0.8178 - auc: 0.7411 - loss: 0.4237 - val_accuracy: 0.8162 - val_auc: 0.7334 - val_loss: 0.4269
Epoch 6/100
3750/3750 - 17s - 4ms/step - accuracy: 0.8183 - auc: 0.7424 - loss: 0.4228 - val_accuracy: 0.8156 - val_auc: 0.7322 - val_loss: 0.4279
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step


In [ ]:
#For some reason there are two model.fit blocks, but it doesn't affect the overall score.
print(duration)

778.6314671039581
